# Clean Output and Obtain Model Results

In [1]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [2]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, f1_score, accuracy_score
import re
import json

## RAG v2 Results

### Step 0: Clean output

In [3]:
import pandas as pd
df = pd.read_csv("/content/drive/Shareddrives/ENGS Final Project/Models/Output/RAW_RESULTS_rag_v2_mistral.csv")

In [ ]:
df['y_pred'].value_counts()

5                                                                                                             258
4                                                                                                             227
3                                                                                                             146
-1                                                                                                            104
2                                                                                                              55
6                                                                                                              39
1                                                                                                              19
Modest Risk                                                                                                    17
-2                                                                                      

In [ ]:
# df[(df['y_pred'] == '-1') | (df['y_pred'] == '-2')]['reasoning'].iloc[3]

In [4]:
# Write a function that extracts answers from the -1 output above
import ast

def extract_answer(input_str):
  json_str_match = re.search(r'\{.*\}', input_str, re.DOTALL)
  if json_str_match:
    json_str = json_str_match.group(0)
    # json_str = json_str.replace("'", "\"")

    try:
        dict_obj = ast.literal_eval(json_str)
        return dict_obj['answer']
    except SyntaxError:
        match = re.search(r'\'answer\':.*', input_str)


        if match:
            # Extract the number from the captured group
            number = match.group(0)
            return number
        else:
            # match = re.search(r'\'answer\':.*\}', input_str)

            # if match:
            #     # Extract the number from the captured group
            #     number = match.group(0)
            #     return number
            # else:
            #     print(json_str)
            return -1 # Error parsing JSON

  else:
      return -2 # No JSON object found

  json_str = json_str_match.group(0)
  dict_obj = ast.literal_eval(json_str)
  return dict_obj['answer']

# df[(df['y_pred'] == '-1')]['reasoning'].apply(extract_answer).value_counts()

df.loc[df['y_pred'] == -1, 'y_pred'] = df.loc[df['y_pred'] == -1, 'reasoning'].apply(extract_answer)

In [ ]:
test = df.loc[df['y_pred'] == -1].apply(extract_answer)

In [5]:
def fix_negative_one_case(row):
  if row['y_pred'] == '-1':
    return extract_answer(row['reasoning'])
  return row['y_pred']

In [6]:
df['y_pred'] = df.apply(fix_negative_one_case, axis=1)

In [ ]:
df['y_pred'].value_counts() # -1s should be fixed!

5                                                                                                             258
4                                                                                                             229
3                                                                                                             147
2                                                                                                              56
6                                                                                                              39
5                                                                                                              23
3                                                                                                              22
Modest Risk                                                                                                    19
1                                                                                       

In [7]:
plaintext_key = {
    'minimal': 1,
    'modest': 2,
    'intermediate': 3,
    'significant': 4,
    'aggressive': 5,
    'highly leveraged': 6
}

In [8]:
def fix_plaintext_case(row):
  if isinstance(row['y_pred'], str):
    for cat in plaintext_key.keys():
      if cat in row['y_pred'].lower():
        return plaintext_key[cat]
    return row['y_pred']

In [ ]:
row = df[df['y_pred']=="'answer': 'Aggressive Risk'}"].iloc[0]

In [ ]:
fix_plaintext_case(row)

5

In [9]:
df['y_pred'] = df.apply(fix_plaintext_case, axis=1)

In [ ]:
df['y_pred'].value_counts()

5                                                                                                             258
4                                                                                                             229
3                                                                                                             147
2                                                                                                              56
6                                                                                                              39
2                                                                                                              21
3                                                                                                              20
1                                                                                                              19
-2                                                                                      

In [10]:
def fix_weird_formatting_case(row):
  if isinstance(row['y_pred'], str):
    if 'answer' in row['y_pred']:
      try:
        res = row['y_pred'].split(':')[-1].replace('}','')
        return int(res)
      except:
        pass
    return row['y_pred']

In [ ]:
row = df[df['y_pred']=="'answer': 4}"].iloc[0]

In [ ]:
fix_weird_formatting_case(row)

4

In [11]:
df['y_pred'] = df.apply(fix_weird_formatting_case, axis=1)

In [ ]:
df['y_pred'].value_counts()

5                                                                                                             258
4                                                                                                             229
3                                                                                                             147
2                                                                                                              56
6                                                                                                              39
1                                                                                                              19
-2                                                                                                             15
5                                                                                                               6
4                                                                                       

In [12]:
def fix_float_case(row):
  if isinstance(row['y_pred'], float):
    return int(row['y_pred'])
  return row['y_pred']

In [13]:
df['y_pred'] = df.apply(fix_weird_formatting_case, axis=1)

In [ ]:
df['y_pred'].value_counts()

5                                                                                                             258
4                                                                                                             229
3                                                                                                             147
2                                                                                                              56
6                                                                                                              39
1                                                                                                              19
-2                                                                                                             15
3.5                                                                                                             3
5.0                                                                                     

In [14]:
def fix_remaining_cases(row):
  if isinstance(row['y_pred'], str):
    if 'Between 4 and 6' in row['y_pred']:
      return 5
    elif 'estimated to be on the higher end of the 1-6 scale' in row['y_pred']:
      return 4
    elif 'Unable to' in row['y_pred']:
      return -2
    elif 'High Financial Risk' in row['y_pred']:
      return 6
    elif "'answer': <A single category from 1-6> } is:" in row['y_pred']:
      return 3 # manually parsed through reasoning to check for this case
  return row['y_pred']

In [ ]:
row = df[df['y_pred']=="'answer': <A single category from 1-6> } is:"].iloc[0]

In [ ]:
row['reasoning']

'# STEP 1 OUTPUT:\n\n   Based on the information provided, the appropriate volatility table to use for Walt Disney Co. (The) is the Medial Volatility table (table 18). This is because the company has a CICRA score of \'3\', which generally applies the medial volatility table. Furthermore, the company operates in the MEDIA & ENTERTAINMENT industry, which is not identified as having low volatility. Therefore, the medial volatility table is the most suitable for this assessment.\n # STEP 2 OUTPUT:\n Based on the provided context, let\'s categorize each financial metric into its corresponding risk category using the Medial Volatility Table:\n\nCore Financial Metrics:\n1. FFO to debt: 0.27 -> Significant Risk (13% - 23%)\n2. Debt to EBITDA: 2.4 -> Intermediate Risk (2.5 - 3.5)\n\nSupplemental financial metrics:\n1. FFO cash interest cover: 6.4 -> Modest Risk (7.5 - 10.5)\n2. EBITDA to interest: 8.28 -> Minimal Risk (greater than or equal to 14)\n3. CFO to debt: 0.36 -> Intermediate Risk (18

In [ ]:
"'answer': <A single category from 1-6> } is:" in row['y_pred']

True

In [15]:
df['y_pred'] = df.apply(fix_remaining_cases, axis=1)

In [ ]:
df['y_pred'].value_counts()

5      258
4      229
3      147
2       56
6       39
1       19
-2      15
3.5      3
-2       2
3.0      2
5.0      2
5        1
4        1
3        1
4.0      1
4.5      1
6        1
Name: y_pred, dtype: int64

In [16]:
def force_int(row): # for when astype doesn't work

  cand = row['y_pred']

  if isinstance(cand, int):
    return cand

  elif isinstance(cand, str):
    cand = float(cand)

  if isinstance(cand, float):
    cand = int(cand)

  return cand

In [17]:
df['y_pred'] = df.apply(force_int, axis=1)

In [ ]:
df['y_pred'].value_counts()

 5.0    261
 4.0    232
 3.0    153
 2.0     56
 6.0     40
 1.0     19
-2.0     17
Name: y_pred, dtype: int64

In [ ]:
df = df[df['y_pred'] != -2] # drop -2 observations

In [ ]:
df['y_pred'].value_counts()

5.0    261
4.0    232
3.0    153
2.0     56
6.0     40
1.0     19
Name: y_pred, dtype: int64

In [18]:
df['y_pred'].astype(float).astype('Int64')

0         3
1         5
2         5
3         4
4         3
       ... 
942       4
943       4
944       5
945       5
946    <NA>
Name: y_pred, Length: 947, dtype: Int64

In [ ]:
df['y_pred'].value_counts()

5.0    261
4.0    232
3.0    153
2.0     56
6.0     40
1.0     19
Name: y_pred, dtype: int64

In [ ]:
df.to_csv("/content/drive/Shareddrives/ENGS Final Project/Models/Output/CLEANED_RESULTS_rag_v2_mistral.csv")

In [ ]:
df.columns

Index(['Unnamed: 0.16', 'Unnamed: 0.15', 'Unnamed: 0.14', 'Unnamed: 0.13',
       'Unnamed: 0.12', 'Unnamed: 0.11', 'Unnamed: 0.10', 'Unnamed: 0.9',
       'Unnamed: 0.8', 'Unnamed: 0.7', 'Unnamed: 0.6', 'Unnamed: 0.5',
       'Unnamed: 0.4', 'Unnamed: 0.3', 'Unnamed: 0.2', 'Unnamed: 0.1',
       'Unnamed: 0', 'company_name', 'sector', 'financial_risk_profile',
       'circa_rating', 'business_description', 'ffo_to_debt_ltm',
       'debt_to_ebitda_ltm', 'cfo_to_debt_ltm', 'focf_to_debt_ltm',
       'dcf_to_debt_ltm', 'ffo_interest_coverage_ltm',
       'ebitda_to_interest_ltm', 'ebit_margin_ltm', 'ebitda_margin_ltm',
       'return_on_capital_ltm', 'y_pred', 'reasoning'],
      dtype='object')

In [19]:
import re
import json

def extract_number_regex(rating):
  match = re.search(r"\[(\d+)\]", rating)
  if match:
    return int(match.group(1))
  else:
    return -1

In [20]:
df['financial_risk_profile_numeric'] = df['financial_risk_profile'].apply(extract_number_regex)

In [46]:
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import accuracy_score, balanced_accuracy_score

In [72]:
df_filtered = df_filtered[~df_filtered['y_pred'].isna()]

In [73]:
df_filtered = df_filtered[df_filtered['y_pred'] >= 0]

In [53]:
df_filtered[:4]

,Unnamed: 0.16,Unnamed: 0.15,Unnamed: 0.14,Unnamed: 0.13,Unnamed: 0.12,Unnamed: 0.11,Unnamed: 0.10,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,...,focf_to_debt_ltm,dcf_to_debt_ltm,ffo_interest_coverage_ltm,ebitda_to_interest_ltm,ebit_margin_ltm,ebitda_margin_ltm,return_on_capital_ltm,y_pred,reasoning,financial_risk_profile_numeric
0,0,0,0,0,0,0,0,0,0,0,...,0.11,0.11,2.93,10.30,7.25,8.70,7.05,3,# STEP 1 OUTPUT:\n\n Based on the informatio...,3
1,1,1,1,1,1,1,1,1,1,1,...,0.12,0.13,2.42,1.47,1.66,4.05,2.03,5,# STEP 1 OUTPUT:\n\n Based on the provided i...,4
2,2,2,2,2,2,2,2,2,2,2,...,0.07,0.14,5.35,8.05,12.20,15.30,8.53,5,# STEP 1 OUTPUT:\n\n Based on the provided i...,4
3,3,3,3,3,3,3,3,3,3,3,...,0.13,0.36,3.20,8.06,8.06,10.00,6.87,4,# STEP 1 OUTPUT:\n\n Based on the provided i...,3


In [43]:
mse = ((df_filtered['y_pred'] - df_filtered['financial_risk_profile_numeric'])**2).mean()
print(f"MSE: {mse:.4f}")

r2 = r2_score(df_filtered['financial_risk_profile_numeric'], df_filtered['y_pred'])
print(f"R-squared: {r2:.4f}")

accuracy = accuracy_score(df_filtered['financial_risk_profile_numeric'], df_filtered['y_pred'])
print(f"Accuracy: {r2:.4f}")

MSE: 2.1432
R-squared: -0.1275
Accuracy: -0.1275


In [48]:
mse = ((df_filtered['y_pred'] - df_filtered['financial_risk_profile_numeric'])**2).mean()
print(f"MSE: {mse:.4f}")

r2 = r2_score(df_filtered['financial_risk_profile_numeric'], df_filtered['y_pred'])
print(f"R-squared: {r2:.4f}")

accuracy = balanced_accuracy_score(df_filtered['financial_risk_profile_numeric'], df_filtered['y_pred'])
print(f"Accuracy: {r2:.4f}")

MSE: 2.1432
R-squared: -0.1275
Accuracy: -0.1275


In [66]:
df_filtered.describe()

,Unnamed: 0.16,Unnamed: 0.15,Unnamed: 0.14,Unnamed: 0.13,Unnamed: 0.12,Unnamed: 0.11,Unnamed: 0.10,Unnamed: 0.9,Unnamed: 0.8,Unnamed: 0.7,...,cfo_to_debt_ltm,focf_to_debt_ltm,dcf_to_debt_ltm,ffo_interest_coverage_ltm,ebitda_to_interest_ltm,ebit_margin_ltm,ebitda_margin_ltm,return_on_capital_ltm,y_pred,financial_risk_profile_numeric
count,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,761.000000,...,738.000000,738.000000,736.000000,722.000000,728.000000,761.000000,760.000000,756.000000,761.000000,761.000000
mean,476.926413,476.926413,476.926413,476.926413,476.926413,476.926413,476.926413,476.926413,476.926413,476.926413,...,0.364295,0.206152,0.364878,17.675194,11.794451,14.738055,23.917855,7.851865,4.024967,3.750329
std,272.257250,272.257250,272.257250,272.257250,272.257250,272.257250,272.257250,272.257250,272.257250,272.257250,...,4.393221,2.655061,4.576378,176.410334,18.457337,15.733060,20.414452,13.328656,1.128883,1.379627
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,-41.830000,-38.720000,-69.780000,-6.040000,0.000000,-56.600000,-54.200000,-42.400000,1.000000,1.000000
25%,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,244.000000,...,0.110000,0.000000,0.030000,2.250000,3.805000,6.050000,10.600000,2.910000,3.000000,3.000000
50%,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,486.000000,...,0.210000,0.120000,0.195000,4.525000,6.195000,12.000000,18.400000,5.980000,4.000000,4.000000
75%,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,706.000000,...,0.420000,0.290000,0.450000,9.452500,12.425000,20.600000,31.225000,10.300000,5.000000,5.000000
max,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,945.000000,...,101.520000,51.120000,85.920000,4683.900000,225.800000,153.100000,182.700000,278.400000,6.000000,6.000000


In [67]:
df_filtered.shape

(761, 35)

In [68]:
df.shape

(947, 35)

In [86]:
correct_count = 0
nans_count = 0
over_count = 0
under_count = 0

for i in range(len(df_filtered)):
  try:
    if int(df.loc[i, "y_pred"]) == int(df.loc[i, "financial_risk_profile_numeric"]):
      correct_count += 1
    elif int(df.loc[i, "y_pred"]) > int(df.loc[i, "financial_risk_profile_numeric"]):
      over_count += 1
    elif int(df.loc[i, "y_pred"]) < int(df.loc[i, "financial_risk_profile_numeric"]):
      under_count += 1
  except Exception as e:
    nans_count += 1

In [87]:
correct_count, nans_count

(153, 131)

In [88]:
correct_count/(len(df_filtered) - nans_count)

0.24285714285714285

In [91]:
over_count/(len(df_filtered) - nans_count)

0.41904761904761906

In [63]:
correct_count = 0

for i, row in df_filtered.iteritems():
  print(i)
  if 'y_pred' in row and int(row['y_pred']) == int(row['financial_risk_profile_numeric']):
    correct_count += 1

Unnamed: 0.16
Unnamed: 0.15
Unnamed: 0.14
Unnamed: 0.13
Unnamed: 0.12
Unnamed: 0.11
Unnamed: 0.10
Unnamed: 0.9
Unnamed: 0.8
Unnamed: 0.7
Unnamed: 0.6
Unnamed: 0.5
Unnamed: 0.4
Unnamed: 0.3
Unnamed: 0.2
Unnamed: 0.1
Unnamed: 0
company_name
sector
financial_risk_profile
circa_rating
business_description
ffo_to_debt_ltm
debt_to_ebitda_ltm
cfo_to_debt_ltm
focf_to_debt_ltm
dcf_to_debt_ltm
ffo_interest_coverage_ltm
ebitda_to_interest_ltm
ebit_margin_ltm
ebitda_margin_ltm
return_on_capital_ltm
y_pred
reasoning
financial_risk_profile_numeric


<ipython-input-63-a636dfb74616>:3: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for i, row in df_filtered.iteritems():


In [58]:
correct_count

0

In [35]:
 df_filtered['financial_risk_profile_numeric'][:10]

0     3
1     4
2     4
3     3
4     3
6     4
7     3
8     4
9     4
10    4
Name: financial_risk_profile_numeric, dtype: int64

In [36]:
 df_filtered['y_pred'][:10]

0     3.0
1     5.0
2     5.0
3     4.0
4     3.0
6     3.0
7     3.0
8     4.0
9     3.0
10    5.0
Name: y_pred, dtype: float64

In [84]:
df_filtered['financial_risk_profile_numeric'].value_counts()

3    224
4    205
6    111
5    101
2     71
1     49
Name: financial_risk_profile_numeric, dtype: int64

In [83]:
df_filtered['y_pred'].value_counts()

5    261
4    232
3    153
2     56
6     40
1     19
Name: y_pred, dtype: int64

In [85]:
153/224

0.6830357142857143

In [38]:
df_filtered['y_pred'] = df_filtered.apply(force_int, axis=1)

In [39]:
def force_int2(row): # for when astype doesn't work

  cand = row['financial_risk_profile_numeric']

  if isinstance(cand, int):
    return cand

  elif isinstance(cand, str):
    cand = float(cand)

  if isinstance(cand, float):
    cand = int(cand)

  return cand

In [40]:
df_filtered['financial_risk_profile_numeric'] = df_filtered.apply(force_int2, axis=1)

In [41]:
df_filtered['financial_risk_profile_numeric'].value_counts()

3    224
4    205
6    111
5    101
2     71
1     49
Name: financial_risk_profile_numeric, dtype: int64

In [42]:
df_filtered['y_pred'].value_counts()

5    261
4    232
3    153
2     56
6     40
1     19
Name: y_pred, dtype: int64

In [ ]:
mse = ((df_filtered['y_pred'] - df_filtered['financial_risk_profile_numeric'])**2).mean()
print(f"MSE: {mse:.4f}")

r2 = r2_score(df_filtered['financial_risk_profile_numeric'], df_filtered['y_pred'])
print(f"R-squared: {r2:.4f}")

accuracy = accuracy_score(df_filtered['financial_risk_profile_numeric'], df_filtered['y_pred'])
print(f"Accuracy: {r2:.4f}")

MSE: 2.1432
R-squared: -0.1275
Accuracy: -0.1275
